In [1]:
# 数据预处理

import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 定义训练集和测试集
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=8, shuffle=True, num_workers=6)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=8, shuffle=False, num_workers=6)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [2]:
# 定义网络

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class ResidualBlock(nn.Module): 
    def __init__(self, inchannel, outchannel, stride=1): 
        super(ResidualBlock, self).__init__() 
        self.left = nn.Sequential( 
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False), 
            nn.BatchNorm2d(outchannel), 
            nn.ReLU(inplace=True), 
            nn.Conv2d(outchannel, outchannel, kernel_size=3,stride=1, padding=1, bias=False), 
            nn.BatchNorm2d(outchannel) 
        ) 
        self.shortcut = nn.Sequential() 
        if stride != 1 or inchannel != outchannel: 
            self.shortcut = nn.Sequential( 
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False), 
                nn.BatchNorm2d(outchannel) 
            ) 
    def forward(self, x): 
        out = self.left(x) 
        out += self.shortcut(x) 
        out = F.relu(out) 
        return out 

class ResNet(nn.Module): 
    def __init__(self, ResidualBlock, num_classes=10): 
        super(ResNet, self).__init__() 
        self.inchannel = 8 
        self.conv1 = nn.Sequential( 
            nn.Conv2d(3, 8, kernel_size=3, stride=1, padding=1, bias=False), 
            nn.BatchNorm2d(8), 
            nn.ReLU(), 
        ) 
        self.layer1 = self.make_layer(ResidualBlock, 8, 2, stride=1) 
        self.layer2 = self.make_layer(ResidualBlock, 16, 2, stride=2) 
        self.layer3 = self.make_layer(ResidualBlock, 32, 2, stride=2) 
        self.layer4 = self.make_layer(ResidualBlock, 64, 2, stride=2) 
        self.fc = nn.Linear(64, num_classes) 
        
    def make_layer(self, block, channels, num_blocks, stride): 
        strides = [stride] + [1] * (num_blocks - 1) #strides=[1,1] 
        layers = [] 
        for stride in strides: 
            layers.append(block(self.inchannel, channels, stride)) 
            self.inchannel = channels 
        return nn.Sequential(*layers) 
    
    def forward(self, x): 
        out = self.conv1(x) 
        out = self.layer1(out) 
        out = self.layer2(out) 
        out = self.layer3(out) 
        out = self.layer4(out) 
        out = F.avg_pool2d(out, 4) 
        out = out.view(out.size(0), -1) 
        out = self.fc(out) 
        return out
    


In [3]:
# 定义损失函数和优化器

import torch.optim as optim

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet14 = ResNet(ResidualBlock).cuda()

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(resnet14.parameters())

In [5]:
# 训练

for epoch in range(40):
    running_loss = 0.0
    
    # 获取数据
    for i, (inputs, labels) in enumerate(trainloader):
        inputs = Variable(inputs).cuda()
        labels = Variable(labels).cuda()
        
        # 梯度归零、前向、反向、参数更新
        optimizer.zero_grad()
        outputs = resnet14(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        #打印输出
        running_loss += loss.data[0]
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
    correct = 0
    total = 0
    for data in testloader:
        images, labels = data
        images = Variable(images).cuda()
        labels = Variable(labels).cuda()
        outputs = resnet14(Variable(images))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
    print('Accuracy of network on the 10000 test images: %d %%' %(100 * correct / total))
            
print('Finished Training')

/home/fh/anaconda3/envs/pytorch/lib/python2.7/site-packages/ipykernel_launcher.py:19: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


[1,  2000] loss: 1.168
[1,  4000] loss: 1.082
[1,  6000] loss: 1.028
Accuracy of network on the 10000 test images: 66 %
[2,  2000] loss: 0.924
[2,  4000] loss: 0.905
[2,  6000] loss: 0.864
Accuracy of network on the 10000 test images: 69 %
[3,  2000] loss: 0.798
[3,  4000] loss: 0.790
[3,  6000] loss: 0.768
Accuracy of network on the 10000 test images: 72 %
[4,  2000] loss: 0.706
[4,  4000] loss: 0.701
[4,  6000] loss: 0.713
Accuracy of network on the 10000 test images: 73 %
[5,  2000] loss: 0.641
[5,  4000] loss: 0.642
[5,  6000] loss: 0.648
Accuracy of network on the 10000 test images: 74 %
[6,  2000] loss: 0.590
[6,  4000] loss: 0.599
[6,  6000] loss: 0.595
Accuracy of network on the 10000 test images: 75 %
[7,  2000] loss: 0.539
[7,  4000] loss: 0.551
[7,  6000] loss: 0.556
Accuracy of network on the 10000 test images: 76 %
[8,  2000] loss: 0.489
[8,  4000] loss: 0.501
[8,  6000] loss: 0.521
Accuracy of network on the 10000 test images: 76 %
[9,  2000] loss: 0.454
[9,  4000] loss: 

Process Process-240:
Process Process-239:
Process Process-237:
Traceback (most recent call last):
Process Process-236:
Process Process-238:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/fh/anaconda3/envs/pytorch/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
  File "/home/fh/anaconda3/envs/pytorch/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
Process Process-235:
Traceback (most recent call last):
    self.run()
Traceback (most recent call last):
  File "/home/fh/anaconda3/envs/pytorch/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/fh/anaconda3/envs/pytorch/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
Traceback (most recent call last):
  File "/home/fh/anaconda3/envs/pytorch/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
    self.run()
  File "/home/fh/anaconda3/envs/pytorch/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap

KeyboardInterrupt: 

KeyboardInterrupt
  File "/home/fh/anaconda3/envs/pytorch/lib/python2.7/multiprocessing/queues.py", line 131, in get
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/home/fh/anaconda3/envs/pytorch/lib/python2.7/multiprocessing/queues.py", line 131, in get
    if not self._poll(timeout):
KeyboardInterrupt
KeyboardInterrupt
    if not self._poll(timeout):
KeyboardInterrupt


In [6]:
# 测试

import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

dataiter = iter(testloader)
images, labels = dataiter.next()
#tmp = np.array(torchvision.utils.make_grid(images))
#cv2.imshow('testImage', tmp)
#print('GroundTruth :', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

outputs = resnet14(Variable(images).cuda())
_, predicted = torch.max(outputs.data, 1)
print('predicted:', ' '.join('%5s' % classes[predicted[j]] for j in range(8)))

correct = 0
total = 0
for data in testloader:
    images, labels = data
    images = Variable(images).cuda()
    labels = Variable(labels).cuda()
    outputs = resnet14(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
print('Accuracy of network on the 10000 test images: %d %%' %(100 * correct / total))

class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
for data in testloader:
    images, labels = data
    images = Variable(images).cuda()
    labels = Variable(labels).cuda()
    outputs = resnet14(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    c = (predicted ==labels).squeeze()
    for i in range(4):
        label = labels[i]
        class_correct[label] += c[i]
        class_total[label] += 1

('predicted:', '  cat  ship  ship plane  frog  frog   car  bird')
Accuracy of network on the 10000 test images: 75 %
